## 1. 필요 라이브러리 불러오기

In [1]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from difflib import SequenceMatcher
okt=Okt()

## 2. 네이버 블로그 크롤링 데이터 가져오기

In [2]:
f_data = pd.read_csv('./list/네이버블로그.csv')
f_data.drop('Unnamed: 0', axis=1, inplace=True)
f_data.dropna(inplace=True)
f_data.set_index('link', inplace=True)

In [3]:
f_data

,title,data
link,,
https://blog.naver.com/thdusrla1677/222433683181,제주요트투어 돌고래투어 차귀도요트,제주도요트투어 / 돌고래투어\n요트투어 / 제주도돌고래투어\n로맨틱하고 럭셔리한 제...
https://blog.naver.com/alsko0o/222339362548,차귀도 요트에서 인생샷 찍기,이번 제주도여행을 떠나서 가족들과 즐거운 시간을 보내고 왔어요 ㅎㅎ\n처음에는 무엇...
https://blog.naver.com/sooki1106/222241198019,꼬마손님의 제주 차귀도요트 즐기기 낚시와 돌고래,차귀도요트를 제대로 즐기는 어제 만난 6살 아이의 이야기를 해볼께요.\n엄마 아빠의...
https://blog.naver.com/sooki1106/222324975043,차귀도요트 쉬는 날 제주사는 즐거움 제주로농원 적채수확,제주사는 즐거움 제주로농원 \n동수씨네 밭에서 적채수확\n봄볕이 따갑게 내리쬐던 날...
https://blog.naver.com/jkssyh/222343155908,제주도 가족여행 바로여기_만족도 최상_차귀도요트,모처럼 제주나들이를 한 남표니와 아들커플과 함께한 차귀도요트\n우리는 16시30분...
...,...,...
https://blog.naver.com/ailysdiary/222325294892,제주요트투어 돌고래투어 힐링여행 차귀도요트,제주요트투어 돌고래투어 힐링여행 차귀도요트\n요즘에는 힐링여행을 다니시는분들이 많습...
https://blog.naver.com/lean_in/222315313105,제주한달살기 Day 8-차귀도요트투어/제주도갯무꽃/미쁜제과/미쁜제과소금빵크로와상/제...,#제주도한달살기의 Day 8의 기록 \n오늘은 바람이 좀 분다.\n차귀도 가는길에 ...
https://blog.naver.com/johanna8772/222251979296,제주 차귀도 요트투어 블루오션 일몰 선셋 감상,제주 차귀도 요트투어\n블루오션 일몰 선셋을\n볼수 있는 멋진 곳이에요.\n제주 차...


## 3. 로드된 파일 twitter 분석기로 명사만 추출

#네이버 블로그 데이터 명사 추출하는 것. 이미 돌려서 따로 파일 저장중이니 구동할 필요x
n=0
result = []
for i in f_data.data:
    n+=1
    if n%1000==0:
        print(n)
    line_s = i.split('\n')

    for line in line_s:
        tokenlist = okt.pos(line, stem=True, norm=True)
        temp=[]
        for word in tokenlist:
            if word[1] in ["Noun"]: # 명사일 때만
                temp.append((word[0])) # 해당 단어를 저장함
        if temp: # 만약 이번에 읽은 데이터에 명사가 존재할 경우에만
            result.append(temp) # 결과에 저장
            
conv_data = np.array(result, dtype=object)
np.save('word_noun_list',conv_data)

## 4. 관광지 명칭과 유사도 측정을 위해 선제적인 관광지 명칭 정제

In [4]:
n_data = pd.read_csv('./list/관광지목록.csv', encoding='cp949')
result = []

for i in n_data.name:
    tokenlist = okt.pos(i, stem=True, norm=True)
    temp=[]
    for word in tokenlist:
        if word[1] in ["Noun"]: # 명사일 때만
            temp.append((word[0])) # 해당 단어를 저장함
    if temp: # 만약 이번에 읽은 데이터에 명사가 존재할 경우에만
        result.append(temp) # 결과에 저장
    else:
        result.append(i)

## 5. (4)에서 진행한 내용을 바탕으로 검색용 단어집 생성

In [5]:
u_list = []
num=0
for i in result:
    try:
        if i[0] == '제주':
            u_list.append(i[1])
        else:
            if i[0] in u_list:
                if len(i[1]) == 1:
                    u_list.append(n_data.name[num][:2])
                else:
                    u_list.append(i[1])
            else:
                if len(i[0]) == 1:
                    u_list.append(n_data.name[num][:2])
                else:
                    u_list.append(i[0])
    except:
        u_list.append(n_data.name[num][:2])
    num+=1

## 6. (5)에서 진행한 검색용 단어집과 선조합된 키워드와 유사도 측정

In [6]:
def dic_maker(list1, list2, list3, list4, name):
    zip_list = {}
    zip_list['장소'] = name
    for i,j in zip(list1, list2):
        zip_list[i] = j
    for i,j in zip(list3, list4):
        zip_list[i] = j
        
    return zip_list

who_with = ['부모님','아이','친구','커플',
            '친척','가족','지인','연인',
            '혼자','둘','셋','넷']

taste = ['경관','인생샷','휴식','공원','유적지','미술','박물관',
         '익스트림','스포츠','체험','드라이브','쇼핑','전시','자연','산책']

room = ['안전인증민박','굿스테이','관광호텔','전통/가족호텔','호스텔/소형호텔',
        '휴양펜션','휴양콘도','일반숙박','생활숙박','농어촌민박','유스호스텔']

r_df = pd.DataFrame()

result = np.load('./list/word_noun_list.npy', allow_pickle=True)
print(result)

from gensim.models import Word2Vec
model = Word2Vec(result, vector_size=300, window=5, min_count=2, workers=8, sg=1, epochs=5)

for name in u_list:
    w_list = []
    t_list = []
    try:
        for para in who_with:
            sim = model.wv.similarity(name, para)
            w_list.append(sim)

        for para in taste:
            sim = model.wv.similarity(name, para)
            t_list.append(sim)

    except:
        print(name)
    r_df = r_df.append(dic_maker(who_with, w_list, taste, t_list, name), ignore_index=True)

[list(['얼마', '전', '지인', '꼭', '법환', '포구', '맛집'])
 list(['역대', '최고', '구성', '샐러드', '이용', '수', '가성', '비', '만점', '곳', '소개'])
 list(['법환', '포구', '돈']) ...
 list(['사진', '찍', '것', '아이', '오기', '아침', '미소', '목장'])
 list(['사진', '글', '엄지', '사진관']) list(['사진', '무단', '사용', '저작권법', '신고'])]
세화
검멀
아끈
하도
본태
한담
성읍
평대
엉또
성널
알뜨
대평
도두
위미
수월
터진
서귀
교래
서귀
문섬
고내
천아
해맞
민오
구좌
생이
서건
성읍
엉알
엉알
원당
조함
5.
검멀
꽃썸
높은
동검
민오
북오
서귀
성읍
어리
큰넓
둔지
모지
민오
빛의
서귀
석부
쇠와
숫모
쉼터
하예
11
9.
개오
고내
구좌
귀덕
당케산물
더 
돛오
돝오
맬튼
문석
방선
삼의
왕이
월대
천아
통오
해거
행원
(주
TH
가메
가파
거믄
거친
걸서
고포
궁대
그랑
김서
까끄
남내
남수
내창
농개
느지
다려
더마
동알
등머을레져
마라
망장
매오
민오
발이
삼밭
샘이
서귀
성읍
성읍
소남
소병
손지
솔동
수악
수월
신대
신산
신산
씨에
알오
알오
연미
영모
예래
예래
예초
림반
용둠
웃세
월나
월라
월라
율랜
임문
조은
족은
족은
족은
질그
추자
취다
캐니
팀케
평대
평대
폴개
풋감
하논
해품
협자
효명
휴림


#파일로 저장하여 따로 사용하였으나, 현재는 이미 파일이 존재하므로 구동할 필요x
r_df.to_csv('모두포함.csv', encoding='utf-8')

## 7. 유사도 측정된 데이터 불러오기

In [7]:
data = pd.read_csv('./list/모두포함.csv', encoding='utf-8')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.dropna(inplace=True)
data.set_index('장소', inplace=True)
data

,부모님,아이,친구,커플,친척,가족,지인,연인,혼자,둘,...,미술,박물관,익스트림,스포츠,체험,드라이브,쇼핑,전시,자연,산책
장소,,,,,,,,,,,,,,,,,,,,,
우도,0.281278,0.278455,0.283144,0.233310,0.145769,0.253947,0.256924,0.287026,0.250177,0.285035,...,0.029967,0.196384,0.311487,0.181149,0.229074,0.296324,0.169707,0.081207,0.264086,0.323481
성산일출봉,0.284983,0.279038,0.249585,0.230316,0.117174,0.268825,0.236047,0.294079,0.288262,0.260949,...,0.162056,0.179847,0.254996,0.141571,0.227539,0.308020,0.196961,0.158528,0.350235,0.384225
사려,0.222917,0.250094,0.282764,0.204979,0.232679,0.260109,0.276297,0.214369,0.295239,0.233492,...,0.150135,0.220628,0.206908,0.157786,0.219424,0.367003,0.313146,0.212586,0.242856,0.338178
카멜리아,0.283482,0.276234,0.250797,0.207405,0.136426,0.306125,0.296107,0.233443,0.223236,0.227095,...,0.106885,0.313480,0.296050,0.164320,0.234763,0.267665,0.263518,0.200841,0.323252,0.305514
협재해수욕장,0.217980,0.299981,0.287009,0.247733,0.185345,0.321046,0.278148,0.262249,0.217425,0.219540,...,0.094250,0.236341,0.222609,0.141542,0.218896,0.372032,0.184556,0.159139,0.246543,0.319210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
숲곶,0.238878,0.274956,0.258249,0.216079,0.176455,0.261070,0.274771,0.276969,0.283284,0.248791,...,0.237461,0.345137,0.283246,0.191717,0.290004,0.340489,0.154053,0.223387,0.298702,0.323404
황금,0.170820,0.152067,0.177315,0.198034,0.122947,0.194686,0.182867,0.179536,0.075795,0.153358,...,0.041369,0.083938,0.100326,0.003157,0.137373,0.164046,0.144878,0.124166,0.175784,0.106879
황우럭,0.336114,0.326635,0.356931,0.237126,0.406536,0.318027,0.289076,0.349832,0.293090,0.280380,...,0.391656,0.315713,0.291308,0.253776,0.307976,0.246671,0.193403,0.332081,0.319304,0.337559


## 8. 데이터 정규화

In [8]:
#정규화해보기
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(data)
data_scaled = scaler.transform(data)
data_df_scaled = pd.DataFrame(data=data_scaled, columns=data.columns, index=data.index)
data_df_scaled

,부모님,아이,친구,커플,친척,가족,지인,연인,혼자,둘,...,미술,박물관,익스트림,스포츠,체험,드라이브,쇼핑,전시,자연,산책
장소,,,,,,,,,,,,,,,,,,,,,
우도,0.525314,0.247828,0.271820,0.555282,0.305442,0.456934,0.375839,0.630470,0.501245,0.565306,...,0.054743,0.175693,0.328885,0.360732,0.380753,0.416342,0.402239,0.128413,0.523041,0.455694
성산일출봉,0.532312,0.248435,0.237730,0.548707,0.244947,0.490936,0.337371,0.646116,0.584897,0.518939,...,0.307495,0.158731,0.273821,0.285168,0.377693,0.437822,0.459158,0.258338,0.742245,0.557846
사려,0.415084,0.218263,0.271433,0.493069,0.489308,0.471016,0.411537,0.469282,0.600221,0.466083,...,0.284685,0.200561,0.226948,0.316126,0.361516,0.546143,0.701817,0.349176,0.469023,0.480409
카멜리아,0.529476,0.245513,0.238961,0.498396,0.285676,0.576179,0.448038,0.511598,0.442070,0.453770,...,0.201925,0.295805,0.313837,0.328602,0.392093,0.363710,0.598168,0.329440,0.673588,0.425478
협재해수욕장,0.405759,0.270267,0.275746,0.586952,0.389170,0.610276,0.414948,0.575503,0.429306,0.439227,...,0.177750,0.216679,0.242253,0.285112,0.360462,0.555379,0.433251,0.259366,0.478404,0.448511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
숲곶,0.445230,0.244180,0.246531,0.517442,0.370361,0.473211,0.408726,0.608158,0.573964,0.495535,...,0.451783,0.328276,0.301357,0.380910,0.502215,0.497451,0.369543,0.367324,0.611120,0.455564
황금,0.316686,0.116075,0.164318,0.477817,0.257162,0.321503,0.239381,0.392007,0.118219,0.311823,...,0.076561,0.060353,0.123059,0.020894,0.197949,0.173417,0.350381,0.200599,0.298361,0.091436
황우럭,0.628885,0.298053,0.346772,0.563660,0.857118,0.603379,0.435084,0.769805,0.595501,0.556346,...,0.746836,0.298095,0.309215,0.499398,0.538042,0.325156,0.451729,0.549968,0.663542,0.479368


## 9. 사용자로부터 받은 키워드를 토대로 정렬

In [9]:
finder = ['연인','둘','드라이브','산책']
f_data = data_df_scaled.sort_values(by=finder, ascending=False)
f_data

,부모님,아이,친구,커플,친척,가족,지인,연인,혼자,둘,...,미술,박물관,익스트림,스포츠,체험,드라이브,쇼핑,전시,자연,산책
장소,,,,,,,,,,,,,,,,,,,,,
추억,0.887833,0.376790,0.423796,0.779969,0.516696,0.832443,0.462665,1.000000,0.620282,0.632611,...,0.370508,0.218484,0.360891,0.456751,0.677768,0.465376,0.647900,0.404113,0.698145,0.540684
시크릿가든,0.477492,0.245186,0.285217,0.818055,0.396239,0.531582,0.419192,0.993663,0.547865,0.476115,...,0.402953,0.239199,0.335793,0.523998,0.396657,0.339870,0.490965,0.471477,0.565647,0.441909
피규,0.754060,0.426949,0.361603,0.810410,0.667383,0.715999,0.451587,0.990048,0.814975,0.739322,...,0.665089,0.358256,0.521167,0.650080,0.823615,0.456310,0.663930,0.646502,0.902127,0.532313
아이,1.000000,1.000000,0.447169,0.852925,0.630265,1.000000,0.455895,0.951488,0.834026,0.696237,...,0.351386,0.174284,0.442663,0.376342,0.763266,0.366346,0.533069,0.444078,0.543651,0.488441
어영,0.699410,0.359300,0.352275,0.712454,0.453018,0.736761,0.642426,0.947642,0.682091,0.614656,...,0.388976,0.288062,0.312253,0.448544,0.629277,0.701569,0.594656,0.320344,0.676030,0.840166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
향교,0.203060,0.044572,0.109593,0.184192,0.388793,0.127767,0.136368,0.115021,0.216497,0.191884,...,0.326971,0.143084,0.049619,0.195987,0.228535,0.273867,0.312409,0.205174,0.214675,0.155218
연안,0.187399,0.077612,0.139189,0.116945,0.285817,0.244035,0.336603,0.093829,0.161561,0.207486,...,0.300421,0.104107,0.175238,0.201889,0.170635,0.087971,0.311741,0.255145,0.353928,0.045834
가마,0.130397,0.092197,0.135988,0.235709,0.135586,0.131960,0.169887,0.085050,0.223434,0.181596,...,0.290674,0.179982,0.138342,0.191583,0.216181,0.124898,0.118738,0.252773,0.348831,0.228354


In [10]:
pure_name = f_data[finder].mean(axis=1).sort_values(ascending=False).head(10).index.to_list()
pure_name

['해비', '어영', '로로', '여행', '안돌', '답다', '신풍', '산책로', '신창', '신창']

In [11]:
all_data = pd.read_excel('./list/수정된파일.xlsx')
all_data.drop('Column1', axis=1, inplace=True)
all_data

,타입,장소,위치,해시태그,좋아요,찜하기,리뷰,일정 등록,방문했어요,조회,...,유아 서비스,유아 서비스 상세,부대시설 기타,Late체크인여부,공항교통편,장애인전용객실여부,유아 서비스 기타,취급품목,취급품목 기타,택배배송여부
0,관광지,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,"['섬속의섬 ', '경관/포토 ', '아이', '맑음']",136,1568,591,0,5,532483,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,관광지,성산일출봉(UNESCO 세계자연유산),제주특별자치도 서귀포시 성산읍 일출로 284-12,"['일출 ', '오름 ', '경관/포토', '부모']",213,2713,586,0,19,461518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,관광지,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"['숲길 ', '걷기/등산 ', '친구', '커플 ', '흐림 ', '봄']",160,2218,501,0,9,590166,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,관광지,카멜리아힐,제주 서귀포시 안덕면 병악로 166,"['경관/포토 ', '커플 ', '아이', '맑음 ', '겨울 ', '우수관광사업체']",127,2295,472,0,14,178391,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,관광지,협재해수욕장,제주특별자치도 제주시 한림읍 한림로 329-10,"['일몰 ', '해수욕장 ', '액티비티', '아이 ', '맑음 ', '여름']",122,1937,452,0,12,208183,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3477,쇼핑,한림민속오일시장,제주특별자치도 제주시 한림읍 한수풀로4길 10,['전통시장'],4,17,0,0,0,10523,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3478,쇼핑,한마음한라봉,제주특별자치도 제주시 애월읍 애월해안로 32,"['특산품 ', '한라봉']",1,17,0,0,0,5718,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3479,쇼핑,한복담,제주특별자치도 제주시 신대로12길 51,"['커플 ', '친구']",0,4,0,0,0,1133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3480,쇼핑,함덕오일시장,제주특별자치도 제주시 조천읍 함덕16길 15-13,['전통시장'],0,6,0,0,0,4676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
only_tour = all_data[all_data['타입']=='관광지']
only_tour = only_tour.loc[:,'타입':'기타 상세']
only_tour

,타입,장소,위치,해시태그,좋아요,찜하기,리뷰,일정 등록,방문했어요,조회,...,상세 정보,이용 시간,요금 정보,장소 특성,주요목적,주요목적 기타,평균 소요 시간,경사도(난이도),편의시설,기타 상세
0,관광지,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,"['섬속의섬 ', '경관/포토 ', '아이', '맑음']",136,1568,591,0,5,532483,...,"우도 외부차량(렌터카, 전세버스) 반입 제한(2022년 7월 31일까지). 단, 1...","평일 : 00:00 ~ 00:00 , 주말 : 00:00 ~ 00:00","유료,일반 입장료 : 1.000원 / 장애인 무료 (선박료 별도)",실내+실외,"체험,포토스팟,기타,등산,물놀이",우도 관광,1~2시간,상,"공용주차장,화장실,편의점,음료대",UNESCO 세계지질공원
1,관광지,성산일출봉(UNESCO 세계자연유산),제주특별자치도 서귀포시 성산읍 일출로 284-12,"['일출 ', '오름 ', '경관/포토', '부모']",213,2713,586,0,19,461518,...,10월~2월: 07:30~19:00 (매표 마감 17:50) / 3월~9월: 07:...,NaN,"유료,개인 (성인 5,000원 / 청소년, 군인 2,500원 / 어린이 2,500원...",실외,등산,산책로. 올레코스. 오름,1시간 미만,상,"공용주차장,현금결제,카드결제,화장실,무료 WIFI,편의점,음료대,유도 및 안내시설,...","UNESCO 세계자연유산,UNESCO 세계지질공원"
2,관광지,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"['숲길 ', '걷기/등산 ', '친구', '커플 ', '흐림 ', '봄']",160,2218,501,0,9,590166,...,09:00~17:00 (17:00 까지 퇴장) / 우천 및 폭설 시 통행 제한 있을...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,화장실,NaN
3,관광지,카멜리아힐,제주 서귀포시 안덕면 병악로 166,"['경관/포토 ', '커플 ', '아이', '맑음 ', '겨울 ', '우수관광사업체']",127,2295,472,0,14,178391,...,동절기(12월~2월) 08:30~17:00 / 하절기(6월~8월) 08:30~18:...,"평일 : 09:00 ~ 18:30 , 주말 : 09:00 ~ 18:30","유료,성인 8.000원 / 청소년 6.000원 / 어린이, 장애인 5.000원",실외,"공연/전시,포토스팟",NaN,2~3시간,중,"공용주차장,현금결제,카드결제,화장실",우수관광사업체
4,관광지,협재해수욕장,제주특별자치도 제주시 한림읍 한림로 329-10,"['일몰 ', '해수욕장 ', '액티비티', '아이 ', '맑음 ', '여름']",122,1937,452,0,12,208183,...,협재 종합상황실: 064-728-3981(해수욕장 개장기간 동안 운영),NaN,NaN,NaN,NaN,NaN,NaN,NaN,"공용주차장,화장실,편의점,음료대,유도 및 안내시설,경보 및 피난시설",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,관광지,황금굴(한림공원),제주특별자치도 제주시 한림읍 한림로 300,"['여름 ', '경관/포토 ', '부모', '아이 ', '친구 ', '동굴']",0,40,0,0,0,8202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,편의점,음료대,유도 및 ...",NaN
1082,관광지,황우럭만화천국사회적협동조합,제주특별자치도 제주시 한림읍 내동남길 7-3,"['친구 ', '아이 ', '체험관광']",0,1,0,0,0,1598,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083,관광지,효명사,제주특별자치도 서귀포시 남원읍 516로 815-41,"['경관/포토 ', '휴식/힐링 ', '절', '사찰']",1,6,0,0,0,5226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1084,관광지,후포해변,제주특별자치도 제주시 추자면 대서5길 87,"['해변 ', '경관/포토']",0,4,0,0,0,2999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
pure_df = pd.DataFrame()
for name in pure_name:   
    df = only_tour[only_tour['장소'].str.contains(name)]
    pure_df = pure_df.append(df)
pure_df

,타입,장소,위치,해시태그,좋아요,찜하기,리뷰,일정 등록,방문했어요,조회,...,상세 정보,이용 시간,요금 정보,장소 특성,주요목적,주요목적 기타,평균 소요 시간,경사도(난이도),편의시설,기타 상세
1070,관광지,해비치컨트리클럽제주,제주특별자치도 서귀포시 남원읍 원님로 399번길 319,"['골프 ', '액티비티 ', '맑음']",0,0,0,0,0,1792,...,"08:00~16:00(04/01~10/31) 예약 신청, 취소, 변경은 유선 전화로...","평일 : 08:00 ~ 18:00 , 주말 : 08:00 ~ 18:00",NaN,NaN,NaN,NaN,NaN,NaN,"공용주차장,현금결제,카드결제,화장실,무료 WIFI,흡연구역,편의점,음료대,유도 및 ...",NaN
861,관광지,어영공원,제주특별자치도 제주시 서해안로 452 (맞은편 공원),"['겯기/등산 ', '경관/포토 ', '휴식/힐링', '테마공원 ', '아이']",0,4,0,0,0,2066,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"공용주차장,화장실",NaN
702,관광지,로로 요가,제주특별자치도 제주시 조천읍 조함해안로 519-10,"['친구 ', '혼자 ', '맑음', '휴식/힐링 ', '요가 ', '패들요가']",1,10,0,0,0,2756,...,비용: 8만 원 (2시간 패들보드+요가),NaN,"유료,2시간 패들보드 + 요가 -8만원",실외,체험,"패들보드, 요가",1~2시간,NaN,NaN,NaN
762,관광지,뽀로로&타요 테마파크 제주,제주특별자치도 서귀포시 안덕면 병악로 269,"['아이 ', '맑음 ', '비.눈', '흐림 ', '테마공원']",0,15,0,0,0,2096,...,NaN,"평일 : 10:00 ~ 18:00 , 주말 : 10:00 ~ 18:00","유료,자유이용권 종일 어른30,000원 아이40,000원 / 자유이용권 After ...",실내+실외,유아시설,NaN,NaN,NaN,NaN,NaN
429,관광지,올레여행자센터,제주특별자치도 서귀포시 중정로 22,"['커플 ', '혼자 ', '친구', '올레']",2,1,2,0,0,5712,...,NaN,"평일 : 08:00 ~ 22:00 , 주말 : 08:00 ~ 22:00",NaN,NaN,NaN,NaN,NaN,NaN,"공용주차장,현금결제,카드결제,화장실,무료 WIFI",NaN
534,관광지,우도스쿠터여행,제주특별자치도 제주시 우도면 2395-5,"['액티비티 ', '친구 ', '커플', '맑음 ', '가을']",0,36,1,0,1,5780,...,NaN,"평일 : 08:00 ~ 19:00 , 주말 : 08:00 ~ 19:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551,관광지,제주스쿠터여행,제주시 서광로 164,"['액티비티 ', '친구 ', '커플', '맑음']",1,18,1,0,0,4211,...,동절기 09:00~18:00,"평일 : 08:00 ~ 19:00 , 주말 : 08:00 ~ 19:00",NaN,NaN,NaN,NaN,NaN,NaN,공용주차장,NaN
1052,관광지,하늘여행 행글라이더체험장,제주특별자치도 서귀포시 안덕면 중산간서로 1877,"['커플 ', '친구 ', '아이', '액티비티']",0,37,0,0,0,9442,...,하절기 19:00 까지 운영,"평일 : 09:00 ~ 18:00 , 주말 : 09:00 ~ 18:00",NaN,NaN,NaN,NaN,NaN,NaN,공용주차장,NaN
270,관광지,안돌오름,제주특별자치도 제주시 구좌읍 송당리 산 66-2,"['부모 ', '커플 ', '혼자', '친구 ', '아이 ', '맑음 ', '오름']",3,30,6,0,0,5668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
351,관광지,답다니탑망대,제주특별자치도 제주시 우도면 연평리 798,"['등대 ', '경관/포토 ', '맑음']",3,18,3,0,0,3380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,공용주차장,NaN


In [14]:
#파일로 저장하여 따로 사용하였으나, 현재는 이미 파일이 존재하므로 구동할 필요x
pure_df.to_csv('top_10.csv', encoding='utf-8')

## 10. 인스타그램 해시태그 게시물 수와 비교를 위한 필요 파일 불러오기

In [15]:
tags = pd.read_excel('./list/jeju_hashtags_instagram_token_Han.xlsx')
tags.drop('Unnamed: 0', axis=1, inplace=True)

tops = pd.read_csv('./list/top_10.csv')
tops.drop('Unnamed: 0', axis=1, inplace=True)

hash_tag = pd.read_excel('./list/jeju_hashtags_instagram_revised.xlsx')
hash_tag.drop('Unnamed: 0', axis=1, inplace=True)
hash_tag

,hashtag,count
0,천지연기정길,6
1,천지연기정길최고,1
2,소남머리,248
3,소남머리해안,8
4,소남머리해변,3
...,...,...
11998,비양도날씨,566
11999,비양도커피숍,256
12000,비양도에서,246
12001,비양도올레카페,132


## 11. top10의 목록과 해시태그와 비교하여 일치하는 태그들의 개시물 수를 더함

In [16]:
abc = pd.DataFrame()

for i in tops.장소:
    co = 0
    for j in hash_tag.hashtag:
        if SequenceMatcher(None, i, j).ratio() > 0.6:
            co += hash_tag[hash_tag.hashtag == j]['count'].values
    abc = abc.append({'name':i,'count':int(co)}, ignore_index=True)
abc = abc.astype({'count':int})

final = pd.merge(tops, abc, right_on='name', left_on='장소').drop('name', axis=1)
final = final.sort_values('count', ascending=False)
final

,타입,장소,위치,해시태그,좋아요,찜하기,리뷰,일정 등록,방문했어요,조회,...,이용 시간,요금 정보,장소 특성,주요목적,주요목적 기타,평균 소요 시간,경사도(난이도),편의시설,기타 상세,count
11,관광지,애월한담해안산책로,제주특별자치도 제주시 애월읍 곽지리 1359,"['해변 ', '걷기/등산 ', '친구', '맑음 ', '유채꽃']",39,851,60,0,1,101409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,435076
5,관광지,우도스쿠터여행,제주특별자치도 제주시 우도면 2395-5,"['액티비티 ', '친구 ', '커플', '맑음 ', '가을']",0,36,1,0,1,5780,...,"평일 : 08:00 ~ 19:00 , 주말 : 08:00 ~ 19:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152550
20,관광지,신창풍차해안도로,제주특별자치도 제주시 한경면 신창리 1322-1,"['커플 ', '경관/포토 ', '해안도로', '커플스냅']",6,151,8,0,8,18951,...,NaN,무료,실외,포토스팟,NaN,1시간 미만,하,NaN,NaN,142195
19,관광지,신창풍차해안도로,제주특별자치도 제주시 한경면 신창리 1322-1,"['커플 ', '경관/포토 ', '해안도로', '커플스냅']",6,151,8,0,8,18951,...,NaN,무료,실외,포토스팟,NaN,1시간 미만,하,NaN,NaN,142195
18,관광지,신창풍차해안도로,제주특별자치도 제주시 한경면 신창리 1322-1,"['커플 ', '경관/포토 ', '해안도로', '커플스냅']",6,151,8,0,8,18951,...,NaN,무료,실외,포토스팟,NaN,1시간 미만,하,NaN,NaN,142195
21,관광지,신창풍차해안도로,제주특별자치도 제주시 한경면 신창리 1322-1,"['커플 ', '경관/포토 ', '해안도로', '커플스냅']",6,151,8,0,8,18951,...,NaN,무료,실외,포토스팟,NaN,1시간 미만,하,NaN,NaN,142195
17,관광지,신창~차귀해안도로,제주특별자치도 제주시 한경면 한경해안로 470,"['일몰 ', '드라이브 ', '커플', '맑음 ', '가을']",20,386,80,0,2,25170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"화장실,편의점,음료대,유도 및 안내시설",NaN,142058
16,관광지,신창~차귀해안도로,제주특별자치도 제주시 한경면 한경해안로 470,"['일몰 ', '드라이브 ', '커플', '맑음 ', '가을']",20,386,80,0,2,25170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"화장실,편의점,음료대,유도 및 안내시설",NaN,142058
15,관광지,신창~차귀해안도로,제주특별자치도 제주시 한경면 한경해안로 470,"['일몰 ', '드라이브 ', '커플', '맑음 ', '가을']",20,386,80,0,2,25170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"화장실,편의점,음료대,유도 및 안내시설",NaN,142058
14,관광지,신창~차귀해안도로,제주특별자치도 제주시 한경면 한경해안로 470,"['일몰 ', '드라이브 ', '커플', '맑음 ', '가을']",20,386,80,0,2,25170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"화장실,편의점,음료대,유도 및 안내시설",NaN,142058


In [17]:
def des(a,b,c,d):
    return a + b*10 + c*0.1 + d
final['가중치'] = final.apply(lambda df: des(df.좋아요, df.찜하기, df.조회, df['count']), axis=1)
final.drop(['좋아요','찜하기','조회','count'], axis=1, inplace=True)

In [18]:
final.drop_duplicates(['장소'], keep='first', inplace=True)
final.set_index('장소', inplace=True)
final.sort_values('가중치', ascending=False)

,타입,위치,해시태그,리뷰,일정 등록,방문했어요,SNS 공유,관광지,음식점,숙박,...,이용 시간,요금 정보,장소 특성,주요목적,주요목적 기타,평균 소요 시간,경사도(난이도),편의시설,기타 상세,가중치
장소,,,,,,,,,,,,,,,,,,,,,
애월한담해안산책로,관광지,제주특별자치도 제주시 애월읍 곽지리 1359,"['해변 ', '걷기/등산 ', '친구', '맑음 ', '유채꽃']",60,0,1,175,"['곽지해수욕장', '한담해변', '애월 한담 카약', '(주)제주해양레저 투명카약...","['봄날카페', '몽상드애월(몽상)', '리치망고', '애월더선셋', '붉은제주',...","['위더스펜션', '해변산책펜션', '애월포구풍경', '씨스테이호텔', '조이랜드펜...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,453765.9
우도스쿠터여행,관광지,제주특별자치도 제주시 우도면 2395-5,"['액티비티 ', '친구 ', '커플', '맑음 ', '가을']",1,0,1,1,"['우도(해양도립공원)', '우도산호해변 홍조단괴 서빈백사', '검멀레해변', '우...","['하하호호(HAHAHOHO)', '섬소나이', '블랑로쉐', '우도 풍원', '봉...","['우도피아', '소섬바당', '우도사랑', '우도쉼팡게스트하우스', '뽀요요펜션'...",...,"평일 : 08:00 ~ 19:00 , 주말 : 08:00 ~ 19:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153488.0
신창~차귀해안도로,관광지,제주특별자치도 제주시 한경면 한경해안로 470,"['일몰 ', '드라이브 ', '커플', '맑음 ', '가을']",80,0,2,22,"['차귀도', '신창풍차해안도로', '싱게물(싱계물)공원', '순례자의교회', '차...","['차귀놀', '세가름', '쓰쓰카페', '차차신창']","['하바나블루스', '노을과원담', '더 프라이빗 시로', '배배게스트하우스', '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"화장실,편의점,음료대,유도 및 안내시설",NaN,148455.0
신창풍차해안도로,관광지,제주특별자치도 제주시 한경면 신창리 1322-1,"['커플 ', '경관/포토 ', '해안도로', '커플스냅']",8,0,8,28,"['신창~차귀해안도로', '싱게물(싱계물)공원', '순례자의교회', '차귀도요트',...","['차귀놀', '세가름', '늘푸른농원 연리지가든', '쓰쓰카페', '차차신창']","['IGH펜션', '하바나블루스', '노을과원담', '더 프라이빗 시로', '배배게...",...,NaN,무료,실외,포토스팟,NaN,1시간 미만,하,NaN,NaN,145606.1
엉알해안산책로,관광지,제주특별자치도 제주시 한경면 고락로,"['일몰 ', '해변 ', '걷기/등산', '커플 ', '맑음 ', '가을']",4,0,0,3,"['당산봉(당오름,차귀오름)', '순례자의교회', '엉알해안', '당오름(한경면)'...","['아찌국밥', '제주본초협동조합', '수월봉 가마솥순대국', '에피소드', '안녕...","['단빌리지게스트하우스', '단추스테이 단추숲', '뿌리게스트하우스']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92773.9
안돌오름,관광지,제주특별자치도 제주시 구좌읍 송당리 산 66-2,"['부모 ', '커플 ', '혼자', '친구 ', '아이 ', '맑음 ', '오름']",6,0,0,5,"['아부오름', '송당승마장', '덕천리 팔자 좋아 길', '송당본향당', '귀빈사...","['풍림다방', '치저스', '천리식당', '제주담']","['제주가좋아서 펜션', '보리게스트하우스', '나무아래독채펜션', '헌치 게스트하...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84895.8
신창리포구,관광지,제주특별자치도 제주시 한경면 신창리,"['포구 ', '해변 ', '문화유적지', '커플']",7,0,1,0,"['신창~차귀해안도로', '신창풍차해안도로', '싱게물(싱계물)공원', '판포포구']","['바다를본돼지(판포점)', '판포분식', '세가름', '늘푸른농원 연리지가든', ...","['IGH펜션', '구름비낭', '하바나블루스', '노을과원담', '더 프라이빗 시...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28710.9
제주스쿠터여행,관광지,제주시 서광로 164,"['액티비티 ', '친구 ', '커플', '맑음']",1,0,0,2,"['용두암', '제주하늘을날다(패러글라이딩)', '용연', '관덕정', '제주목관아...","['제주김만복(만복이네김밥집)', '올래국수', '미친부엌', '고집돌우럭(제주공항...","['메종글래드 제주', '라마다프라자제주호텔', '제주bp게스트하우스 JEJU BP...",...,"평일 : 08:00 ~ 19:00 , 주말 : 08:00 ~ 19:00",NaN,NaN,NaN,NaN,NaN,NaN,공용주차장,NaN,17506.1
뽀로로&타요 테마파크 제주,관광지,제주특별자치도 서귀포시 안덕면 병악로 269,"['아이 ', '맑음 ', '비.눈', '흐림 ', '테마공원']",0,0,0,1,"['카멜리아힐', '아라고나이트 온천(디아넥스호텔)', '방주교회', '본태박물관'...","['아뜰리에 제주명월', '숨비나리카페', '포도호텔레스토랑', '숨비나리']","['포도호텔', '디아넥스 호텔', '루체빌리조트', '테디밸리호텔']",...,"평일 : 10:00 ~ 18:00 , 주말 : 10:00 ~ 18:00","유료,자유이용권 종일 어른30,000원 아이40,000원 / 자유이용권 After ...",실내+실외,유아시설,NaN,NaN,NaN,NaN,NaN,4289.6


## 12. 데이터프레임 가공을 위한 함수

In [19]:
def to_string(arr):
    ml = arr.replace(' ','').split("'")
    if '[' in ml:
        ml.remove('[')
    if ']' in ml:
        ml.remove(']')
    while ',' in ml:
        ml.remove(',')
    string = ''
    for i in ml:
        string = string+' '+i
    return string

def to_list(arr):
    ml = arr.replace(' ','').split("'")
    if '[' in ml:
        ml.remove('[')
    if ']' in ml:
        ml.remove(']')
    while ',' in ml:
        ml.remove(',')
    return ml

## 13. final 데이터프레임 가공

In [20]:
final['관광지'] = final.apply(lambda val:to_list(val.관광지), axis=1)
final['음식점'] = final.apply(lambda val:to_list(val.음식점), axis=1)
final['숙박'] = final.apply(lambda val:to_list(val.숙박), axis=1)
final

,타입,위치,해시태그,리뷰,일정 등록,방문했어요,SNS 공유,관광지,음식점,숙박,...,이용 시간,요금 정보,장소 특성,주요목적,주요목적 기타,평균 소요 시간,경사도(난이도),편의시설,기타 상세,가중치
장소,,,,,,,,,,,,,,,,,,,,,
애월한담해안산책로,관광지,제주특별자치도 제주시 애월읍 곽지리 1359,"['해변 ', '걷기/등산 ', '친구', '맑음 ', '유채꽃']",60,0,1,175,"[곽지해수욕장, 한담해변, 애월한담카약, (주)제주해양레저투명카약, 제주올레16코스...","[봄날카페, 몽상드애월(몽상), 리치망고, 애월더선셋, 붉은제주, 고불락, 카페콜라...","[위더스펜션, 해변산책펜션, 애월포구풍경, 씨스테이호텔, 조이랜드펜션, 하쿠나마타타...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,453765.9
우도스쿠터여행,관광지,제주특별자치도 제주시 우도면 2395-5,"['액티비티 ', '친구 ', '커플', '맑음 ', '가을']",1,0,1,1,"[우도(해양도립공원), 우도산호해변홍조단괴서빈백사, 검멀레해변, 우도비양도, 제주올...","[하하호호(HAHAHOHO), 섬소나이, 블랑로쉐, 우도풍원, 봉끄랑, 우도해녀식당...","[우도피아, 소섬바당, 우도사랑, 우도쉼팡게스트하우스, 뽀요요펜션, 해오름민박, 하...",...,"평일 : 08:00 ~ 19:00 , 주말 : 08:00 ~ 19:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153488.0
신창풍차해안도로,관광지,제주특별자치도 제주시 한경면 신창리 1322-1,"['커플 ', '경관/포토 ', '해안도로', '커플스냅']",8,0,8,28,"[신창~차귀해안도로, 싱게물(싱계물)공원, 순례자의교회, 차귀도요트, 절부암, 성김...","[차귀놀, 세가름, 늘푸른농원연리지가든, 쓰쓰카페, 차차신창]","[IGH펜션, 하바나블루스, 노을과원담, 더프라이빗시로, 배배게스트하우스, 바사팬스...",...,NaN,무료,실외,포토스팟,NaN,1시간 미만,하,NaN,NaN,145606.1
신창~차귀해안도로,관광지,제주특별자치도 제주시 한경면 한경해안로 470,"['일몰 ', '드라이브 ', '커플', '맑음 ', '가을']",80,0,2,22,"[차귀도, 신창풍차해안도로, 싱게물(싱계물)공원, 순례자의교회, 차귀도요트, 생이기...","[차귀놀, 세가름, 쓰쓰카페, 차차신창]","[하바나블루스, 노을과원담, 더프라이빗시로, 배배게스트하우스, 바사팬스파, 뿌리게스...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"화장실,편의점,음료대,유도 및 안내시설",NaN,148455.0
엉알해안산책로,관광지,제주특별자치도 제주시 한경면 고락로,"['일몰 ', '해변 ', '걷기/등산', '커플 ', '맑음 ', '가을']",4,0,0,3,"[당산봉(당오름,차귀오름), 순례자의교회, 엉알해안, 당오름(한경면), 천주교순례길...","[아찌국밥, 제주본초협동조합, 수월봉가마솥순대국, 에피소드, 안녕구움]","[단빌리지게스트하우스, 단추스테이단추숲, 뿌리게스트하우스]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92773.9
안돌오름,관광지,제주특별자치도 제주시 구좌읍 송당리 산 66-2,"['부모 ', '커플 ', '혼자', '친구 ', '아이 ', '맑음 ', '오름']",6,0,0,5,"[아부오름, 송당승마장, 덕천리팔자좋아길, 송당본향당, 귀빈사, 민오름(구좌읍), ...","[풍림다방, 치저스, 천리식당, 제주담]","[제주가좋아서펜션, 보리게스트하우스, 나무아래독채펜션, 헌치게스트하우스]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84895.8
신창리포구,관광지,제주특별자치도 제주시 한경면 신창리,"['포구 ', '해변 ', '문화유적지', '커플']",7,0,1,0,"[신창~차귀해안도로, 신창풍차해안도로, 싱게물(싱계물)공원, 판포포구]","[바다를본돼지(판포점), 판포분식, 세가름, 늘푸른농원연리지가든, 쓰쓰카페, 차차신창]","[IGH펜션, 구름비낭, 하바나블루스, 노을과원담, 더프라이빗시로, 배배게스트하우스...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28710.9
제주스쿠터여행,관광지,제주시 서광로 164,"['액티비티 ', '친구 ', '커플', '맑음']",1,0,0,2,"[용두암, 제주하늘을날다(패러글라이딩), 용연, 관덕정, 제주목관아, 누웨마루거리(...","[제주김만복(만복이네김밥집), 올래국수, 미친부엌, 고집돌우럭(제주공항점), 우진해...","[메종글래드제주, 라마다프라자제주호텔, 제주bp게스트하우스JEJUBPGUESTHOU...",...,"평일 : 08:00 ~ 19:00 , 주말 : 08:00 ~ 19:00",NaN,NaN,NaN,NaN,NaN,NaN,공용주차장,NaN,17506.1
뽀로로&타요 테마파크 제주,관광지,제주특별자치도 서귀포시 안덕면 병악로 269,"['아이 ', '맑음 ', '비.눈', '흐림 ', '테마공원']",0,0,0,1,"[카멜리아힐, 아라고나이트온천(디아넥스호텔), 방주교회, 본태박물관, 헬로키티아일랜...","[아뜰리에제주명월, 숨비나리카페, 포도호텔레스토랑, 숨비나리]","[포도호텔, 디아넥스호텔, 루체빌리조트, 테디밸리호텔]",...,"평일 : 10:00 ~ 18:00 , 주말 : 10:00 ~ 18:00","유료,자유이용권 종일 어른30,000원 아이40,000원 / 자유이용권 After ...",실내+실외,유아시설,NaN,NaN,NaN,NaN,NaN,4289.6


## 14-1. 사용자가 원하는 항목의 데이터의 음식점 항목 리스트

In [21]:
l_data = final['음식점'][0]
l_data

['봄날카페',
 '몽상드애월(몽상)',
 '리치망고',
 '애월더선셋',
 '붉은제주',
 '고불락',
 '카페콜라',
 '호박다방',
 'ROAD129(로드129)',
 '지금이순간',
 '숙이네보리빵',
 'jival지발',
 '곽지박사장&김사장네',
 '은혜전복',
 '무인카페산책',
 '제주샘주영농협동조합',
 '카페태희',
 '곽지식당',
 '해품',
 '라스또르따스',
 '꽁순이네',
 '탐라면',
 '몬스터살롱',
 '파리바게트애월',
 '카페하물',
 '카페소바니',
 '백번가든',
 '달에',
 '미인계카페',
 '마레벤또',
 '수타명가',
 '마틸다',
 '귤따고레드향갈고',
 '하우스레서피',
 '곽지BAR다',
 '애월리안',
 '메리앤폴',
 '반딧불한담',
 '쿡이야',
 '산책하우스',
 '로맨틱하드보일드',
 '맛있는여가생활',
 '안트레',
 '귀덕간식',
 '인디언키친',
 '애월고깃간']

## 14-2. 찾고자 하는 음식점 데이터 불러오기

In [22]:
df = pd.read_excel('./list/수정된파일.xlsx')
df.drop('Column1', axis=1, inplace=True)
df = df[df['타입'] == '음식점']

df['cate_mix'] = df.apply(lambda val:to_string(val.해시태그), axis=1)
df['해시태그'] = df.apply(lambda val:to_list(val.해시태그), axis=1)
df.set_index('장소', inplace=True)
df.reset_index(inplace=True)

df

,장소,타입,위치,해시태그,좋아요,찜하기,리뷰,일정 등록,방문했어요,조회,...,유아 서비스 상세,부대시설 기타,Late체크인여부,공항교통편,장애인전용객실여부,유아 서비스 기타,취급품목,취급품목 기타,택배배송여부,cate_mix
0,휴림,음식점,제주특별자치도 제주시 애월읍 광령남서길 40,"[부모, 친구, 아이, 커플, 액티비티, 야영장, 캠핑장]",0,3,0,0,0,8566,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,부모 친구 아이 커플 액티비티 야영장 캠핑장
1,봄날카페,음식점,제주특별자치도 제주시 애월읍 애월로1길 25,"[카페, 커피, 음료]",43,558,141,0,2,56839,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,카페 커피 음료
2,명진전복,음식점,제주특별자치도 제주시 구좌읍 해맞이해안로 1282,"[전복, 돌솥밥, 회]",27,537,66,0,0,43587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,전복 돌솥밥 회
3,몽상드애월(몽상),음식점,제주특별자치도 제주시 애월읍 애월북서길 56-1,"[카페, 커피, 음료, 디저트]",18,421,66,0,1,39458,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,카페 커피 음료 디저트
4,서연의집(영화 건축학개론),음식점,제주특별자치도 서귀포시 남원읍 위미해안로 86,"[카페, 커피, 아메리카노]",14,218,66,0,1,34936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,카페 커피 아메리카노
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331,황금어장(제주시),음식점,제주특별자치도 제주시 사장길 14,"[향토음식, 옥돔미역국, 갈치호박국, 성게국, 방어회]",1,7,0,0,0,4289,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,향토음식 옥돔미역국 갈치호박국 성게국 방어회
1332,황원갈비,음식점,제주특별자치도 제주시 구좌읍 김녕로 129,"[갈비, 막창구이, 갈비탕, 육개장]",1,3,0,0,0,850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,갈비 막창구이 갈비탕 육개장
1333,후년하우스,음식점,제주특별자치도 제주시 삼봉로 12,"[소고기샤브샤브, 돼지고기샤브샤브, 오리고기샤브샤브, 말고기샤브샤브, 샤브샤브]",0,10,0,0,0,1157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,소고기샤브샤브 돼지고기샤브샤브 오리고기샤브샤브 말고기샤브샤브 샤브샤브
1334,흑돼지돈까스 뷔페 밀라네사,음식점,제주특별자치도 제주시 매촌2길 78 (도련1동) 어베인건물 1층,"[흑돼지, 돈까스, 뷔페, 흑돼지돈까스, 키즈존, 놀이방]",1,1,0,0,0,1342,...,수유실,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,흑돼지 돈까스 뷔페 흑돼지돈까스 키즈존 놀이방


## 14-3. 사용자가 원하는 음식점 키워드

In [23]:
def counter(arr):
    user_cok_list = ['카페','커피','음료']
    count = 0
    for i in arr:
        if i in user_cok_list:
            count+=1
    return count

In [24]:
df['val_count'] = df.apply(lambda val:counter(val.해시태그), axis=1)
cok_arr = df.sort_values('val_count',ascending=False)['장소'].values
cok_arr

array(['로치아 커피로스터스', '미인계 카페', '쓰담뜨담', ..., '모리노아루요', '메리앤폴', '흑염소마루'],
      dtype=object)

## 14-4. 사용자가 원하는 음식점 출력

In [25]:
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도

count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df['cate_mix'])
place_simi_cate = cosine_similarity(place_category, place_category)
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

def find_simi_place(df, sorted_ind, place_name, top_n):
    place_title = df[df['장소'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

for name in cok_arr[:50]:
    n_list = find_simi_place(df, place_simi_cate_sorted_ind, name, 6)
    find_list = n_list.장소.values
    
    if find_list[0] in l_data:
        break
    
n_list[:1]

,장소,타입,위치,해시태그,좋아요,찜하기,리뷰,일정 등록,방문했어요,조회,...,부대시설 기타,Late체크인여부,공항교통편,장애인전용객실여부,유아 서비스 기타,취급품목,취급품목 기타,택배배송여부,cate_mix,val_count
1044,안트레,음식점,제주특별자치도 제주시 애월읍 애월로 11,"[카페, 커피, 음료, 당근케이크]",0,5,0,0,0,655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,카페 커피 음료 당근케이크,3


## 15-1. 사용자가 원하는 항목의 데이터의 숙소 항목 리스트

In [26]:
l_data = final['숙박'][0]
l_data

['위더스펜션',
 '해변산책펜션',
 '애월포구풍경',
 '씨스테이호텔',
 '조이랜드펜션',
 '하쿠나마타타게스트하우스',
 '아이랑키즈펜션',
 '고불락하우스',
 '탱자탱자게스트하우스',
 '오누게스트하우스',
 '토리코티지카레클린트',
 '마눌펜션',
 '발로바쉬',
 '모물펜션',
 '제주토비스콘도',
 '한담풍경하우스',
 '오후여섯시펜션',
 '바다와자전거']

## 15-2. 찾고자 하는 숙소 데이터 불러오기

In [27]:
df = pd.read_excel('./list/수정된파일.xlsx')
df.drop('Column1', axis=1, inplace=True)
df = df[df['타입'] == '숙박']

df['cate_mix'] = df.apply(lambda val:to_string(val.해시태그), axis=1)
df['해시태그'] = df.apply(lambda val:to_list(val.해시태그), axis=1)
df.set_index('장소', inplace=True)
df.reset_index(inplace=True)

df

,장소,타입,위치,해시태그,좋아요,찜하기,리뷰,일정 등록,방문했어요,조회,...,유아 서비스 상세,부대시설 기타,Late체크인여부,공항교통편,장애인전용객실여부,유아 서비스 기타,취급품목,취급품목 기타,택배배송여부,cate_mix
0,나미송 머무는 곳,숙박,제주특별자치도 제주시 애월읍 평화로 2476,[민박],21,22,133,0,37,29041,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,민박
1,해비치호텔앤드리조트,숙박,제주특별자치도 서귀포시 표선면 민속해안로 537,[휴식/치유],14,57,45,0,1,32372,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,휴식/치유
2,신라호텔제주,숙박,제주도 서귀포시 중문관광로 72번길 75,"[최고급, VIP, 중문, 신라, 중문관광단지]",7,47,43,0,0,21519,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,최고급 VIP 중문 신라 중문관광단지
3,롯데호텔 제주,숙박,제주특별자치도 서귀포시 중문관광로72번길 35,"[호텔, 5성급, 중문, 리조트]",9,40,38,0,1,21976,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,호텔 5성급 중문 리조트
4,제주조천스위스마을게스트하우스,숙박,제주특별자치도 제주시 조천읍 함와로 566-27,[숙박],11,46,29,0,0,46146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,숙박
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,황재여관,숙박,제주특별자치도 제주시 추자면 추자로 70,"[추자항, 추자숙소, 추자도]",0,0,0,0,0,288,...,NaN,NaN,가능,NaN,없음,NaN,NaN,NaN,NaN,추자항 추자숙소 추자도
930,훼밀리민박,숙박,제주특별자치도 서귀포시 안덕면 형제해안로 24,[휴식],1,0,0,0,0,2083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,휴식
931,휴마루리조트,숙박,제주특별자치도 서귀포시 중산간동로8183번길 16,[휴식/치유],1,1,0,0,0,1693,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,휴식/치유
932,힐링캠프펜션,숙박,서귀포시 남원읍 위미항구로 96,[[]],0,0,0,0,0,630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


## 15-3. 사용자가 원하는 숙소 키워드

In [28]:
def counter2(arr):
    user_cok_list = ['휴식 ', '애월 ', '게스트하우스']
    count = 0
    for i in arr:
        if i in user_cok_list:
            count+=1
    return count

In [29]:
df['val_count'] = df.apply(lambda val:counter2(val.해시태그), axis=1)
cok_arr = df.sort_values('val_count',ascending=False)['장소'].values
cok_arr

array(['쿨쿨하우스', '단빌리지게스트하우스', '소섬바당', '레드우드하우스', '뒹글하우스', '김군아미고게스트하우스',
       '라메종베니in제주', '단추스테이 단추숲', '금서방네이층집', '알랑가게스트하우스', '솔로 게스트하우스',
       '엘마르게스트하우스', '여울목게스트하우스(월정리)', '레몬트리게스트하우스', '하마다 게스트하우스',
       '우리희망이 게스트하우스', '핫플게스트하우스', '예스준게스트하우스', '언니네여인숙',
       '솔로게스트하우스 앤 두루두루펜션', '오름게스트하우스', '탐탐56', '도시락게스트하우스', '또랑게스트하우스',
       '게스트하우스 클럽 제이제이', '콤포스텔라 별들의들판', '레프트핸더 게스트하우스', '신산게스트하우스',
       '루미수다게스트하우스', '탱자탱자게스트하우스', '나무이야기', '스쿠버스토리', '노닐다 게스트하우스',
       '안녕프로젝트 게스트하우스', '나무이야기게스트하우스 3호점', '수상한집 광보네', '나무이야기게스트하우스 2호점',
       '동복리671', '블랙롯지', '피우다 게스트하우스', '와락 게스트하우스', '꽃하루방게스트하우스',
       '흰고래게스트하우스', 'ZeZuZip', '제주도푸른산푸른바다', '우리두리게스트하우스', '나날 게스트하우스',
       '헌치 게스트하우스', '수상한소금밭게스트하우스', '배배게스트하우스', '바랑쉬게스트하우스', '오빌하우스',
       '우도쉼팡게스트하우스', '밧돌 게스트하우스', '볼몽지엥하우스', '아침해변', '아쿠아뷰티크', '아템파우제',
       '아토하우스', '월정소랑', '아트모텔', '안녕김녕sea', '아침의향기', '안트레 펜션', '우연히 행복해지다',
       '알로하민박', '알앤비타운', '우도올레펜션', '월정플레이스', '오비두스', '아침의 새소리', '웨스티하우스',
       '씨제이 관광호텔', '아

## 15-4. 사용자가 원하는 음식점 출력

In [30]:
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도

count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df['cate_mix'])
place_simi_cate = cosine_similarity(place_category, place_category)
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

def find_simi_place(df, sorted_ind, place_name, top_n):
    place_title = df[df['장소'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

for name in cok_arr[:50]:
    n_list = find_simi_place(df, place_simi_cate_sorted_ind, name, 6)
    find_list = n_list.장소.values
    
    if find_list[0] in l_data:
        break
        
n_list[:1]

,장소,타입,위치,해시태그,좋아요,찜하기,리뷰,일정 등록,방문했어요,조회,...,부대시설 기타,Late체크인여부,공항교통편,장애인전용객실여부,유아 서비스 기타,취급품목,취급품목 기타,택배배송여부,cate_mix,val_count
827,탱자탱자게스트하우스,숙박,제주특별자치도 제주시 애월읍 애월로9길 69,"[휴식, 애월, 게스트하우스]",0,0,0,0,0,1301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,휴식 애월 게스트하우스,1
